# Script #1 - Convert NetCDF Imagery to Calculated Rasters

*Mike Huff, 2025*

https://github.com/m-huff

This script takes an input folder, which should contain a set of NetCDF files (all the same dimensions - this script is intended to process NASA GOES satellite imagery), and joins them into TWO combined GEOTIFF rasters, one that contains the sum of all optical energy from each processed NetCDF file and one that contains the mean value of all flashes above a threshold (which is meant to screen out the bottom percentage of values that are not actual lightning flashes, but rather just optical "noise").

This also includes a value transformer that converts the very small optical energy values to scaled values that will fit within a raster image in GIS nicely. To edit the scale of values this script produces, change the "THRESHOLD" variable in the script.

To use this script, point the DATA_DIR and OUTPUT_DIR variables at a folder filled with your NetCDF files and an empty output folder that will contain the processed rasters, respectively.

The "SUFFIX" variable is currently blank, but can be used to add a suffix to the output files if you intend to run this script multiple times.

Note that this script may take several minutes to hours depending on the number of NetCDF files processed. In my case, I used approximately 260,000 NetCDF files and this script took 24+ hours to complete.

In [ ]:
%pip install rasterio matplotlib netCDF4 collections os glob numpy

import os
import glob
import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from collections import Counter

### VARIABLES TO CONTROL THE SCRIPT
### THESE ARE DESCRIBED IN THE MARKDOWN CELL ABOVE
DATA_DIR = r"E:\GOES-R Lightning Data\EAST-2025"
OUTPUT_DIR = r"E:\GOES-R Lightning Data\EAST-RASTERS"
PIXEL_SCALE = 1e5
THRESHOLD = 1.2e-05
FILE_SUFFIX = ""

max_raster = None

files = glob.glob(os.path.join(DATA_DIR, "*.nc"))

for i, f in enumerate(files, start=1):
    with Dataset(f, "r") as ds:

        energy = ds.variables["Total_Optical_energy"][:].filled(0)
        if np.all(energy == 0):
            continue

        if max_raster is None:
            shape = energy.shape
            max_raster = np.zeros(shape, dtype=np.float64)
            mean_sum = np.zeros(shape, dtype=np.float64)
            mean_count = np.zeros(shape, dtype=np.float64)

        max_raster = np.maximum(max_raster, energy)
        mask = energy > THRESHOLD
        mean_sum += np.where(mask, energy, 0)
        mean_count += mask.astype(np.float64)

mean_raster = np.divide(mean_sum, mean_count, out=np.zeros_like(mean_sum), where=mean_count > 0)
max_raster *= PIXEL_SCALE
mean_raster *= PIXEL_SCALE
os.makedirs(OUTPUT_DIR, exist_ok=True)

def save_raster(arr, name):
    out_path = os.path.join(OUTPUT_DIR, f"{name}.tif")
    height, width = arr.shape
    transform = from_origin(0, 0, 1, 1)
    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype="float32",
        crs=None,
        transform=transform,
    ) as dst:
        dst.write(arr.astype(np.float32), 1)

save_raster(max_raster, f"max_energy{FILE_SUFFIX}")
save_raster(mean_raster, f"mean_energy{FILE_SUFFIX}")
print("Script finished.")
